In [ ]:
# Cell 1: Imports and Paths
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch
from sklearn.covariance import LedoitWolf
import os
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Define paths
data_path = r'C:/Users/lucas/OneDrive/Bureau/HRP/DATA (CRSP)/PREPROCESSED DATA/ADA-HRP-Preprocessed-DATA.csv'
rolling_dir = r'C:/Users/lucas/OneDrive/Bureau/HRP/DATA (CRSP)/Rolling Windows'
os.makedirs(rolling_dir, exist_ok=True)

In [ ]:
# Cell 2: Load Data and Prepare Dates (Corrected Sorting)
# Load the data
df = pd.read_csv(data_path)

# Identify date columns
date_cols = [col for col in df.columns if col not in ['PERMNO', 'Company_Ticker']]

# Parse mangled column names to dates for sorting
parsed_strs = [col.replace('_', ':') for col in date_cols]
parsed_dates = pd.to_datetime(parsed_strs, errors='coerce')

# Sort by parsed dates
sort_order = np.argsort(parsed_dates)
date_cols = [date_cols[i] for i in sort_order]
dates = parsed_dates[sort_order]
date_strs = [d.strftime('%Y-%m-%d') for d in dates]

# Convert date columns to numeric
for col in date_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Filter to stock rows only (exclude benchmarks)
stocks_df = df[df['PERMNO'].notna()].copy()

# Function to get quarterly end dates
def get_quarterly_dates(dates):
    quarterly_dates = []
    df_dates = pd.DataFrame({'date': dates})
    df_dates['year'] = df_dates['date'].dt.year
    df_dates['quarter'] = df_dates['date'].dt.quarter
    quarterly_ends = df_dates.groupby(['year', 'quarter'])['date'].max()
    return quarterly_ends.tolist()

quarterly_rebalance_dates = get_quarterly_dates(dates)
print(f"Total quarterly dates available: {len(quarterly_rebalance_dates)}")

In [ ]:
# Cell 3: HRP Functions
def get_correlation_distance(corr):
    return ((1 - corr) / 2) ** 0.5

def get_quasi_diag(link):
    link = link.astype(int)
    sort_ix = pd.Series([link[-1, 0], link[-1, 1]])
    num_items = link[-1, 3]
    while sort_ix.max() >= num_items:
        sort_ix.index = range(0, sort_ix.shape[0] * 2, 2)
        df0 = sort_ix[sort_ix >= num_items]
        i = df0.index
        j = df0.values - num_items
        sort_ix[i] = link[j, 0]
        df0 = pd.Series(link[j, 1], index=i + 1)
        sort_ix = pd.concat([sort_ix, df0])
        sort_ix = sort_ix.sort_index()
        sort_ix.index = range(sort_ix.shape[0])
    return sort_ix.tolist()

def get_cluster_var(cov, c_items):
    cov_ = cov.loc[c_items, c_items]
    ivp = 1 / np.diag(cov_)
    ivp /= ivp.sum()
    return (ivp @ cov_ @ ivp)

def get_recursive_bisection(cov, sort_ix):
    w = pd.Series(1.0, index=sort_ix)
    c_items = [sort_ix]
    while len(c_items) > 0:
        c_items = [i[j:k] for i in c_items for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) if len(i) > 1]
        for i in range(0, len(c_items), 2):
            c_items0 = c_items[i]
            c_items1 = c_items[i + 1]
            c_var0 = get_cluster_var(cov, c_items0)
            c_var1 = get_cluster_var(cov, c_items1)
            alpha = 1 - c_var0 / (c_var0 + c_var1)
            w[c_items0] *= alpha
            w[c_items1] *= 1 - alpha
    # Normalize to ensure weights sum to exactly 1.0 (fix numerical drift)
    w = w / w.sum()
    return w

In [ ]:
# Cell 4: Process ONE RANDOM Quarterly Rebalance (QuickTest)
from sklearn.metrics.pairwise import euclidean_distances

# Optimized Euclidean distance function
def get_euclidean_distance(dist):
    return pd.DataFrame(euclidean_distances(dist), index=dist.index, columns=dist.columns)

# Select ONE random quarterly date
random.seed(42)  # For reproducibility, remove this line for truly random selection
rebal_date = random.choice(quarterly_rebalance_dates)
print(f"\n{'='*60}")
print(f"QUICK TEST - Processing ONE random window")
print(f"Selected rebalance date: {rebal_date.strftime('%Y-%m-%d')}")
print(f"{'='*60}\n")

rebal_str = rebal_date.strftime('%Y-%m-%d')

# Find the start of the 12-month lookback
start_date = rebal_date - relativedelta(months=11)  # 12 periods including rebal_date
window_dates = [d for d in dates if start_date <= d <= rebal_date]

if len(window_dates) < 12:
    print(f"ERROR: Insufficient data for 12-month window (only {len(window_dates)} periods)")
else:
    print(f"Window period: {window_dates[0].strftime('%Y-%m-%d')} to {window_dates[-1].strftime('%Y-%m-%d')}")
    print(f"Number of periods in window: {len(window_dates)}")
    
    window_strs = [d.strftime('%Y-%m-%d') for d in window_dates]
    
    # Map to actual mangled column names
    window_indices = [date_strs.index(s) for s in window_strs]
    actual_window_cols = [date_cols[i] for i in window_indices]
    
    # Select window data
    window_df = stocks_df[['PERMNO', 'Company_Ticker'] + actual_window_cols].copy()
    
    # Drop rows with NaN Company_Ticker if any
    window_df = window_df[window_df['Company_Ticker'].notna()]
    
    # Filter stocks with at least 12 valid (non-NaN) values in the window
    valid_mask = window_df[actual_window_cols].notna().sum(axis=1) >= 12
    window_df = window_df[valid_mask]
    
    print(f"Number of stocks with sufficient data: {len(window_df)}")
    
    if len(window_df) < 2:
        print(f"ERROR: Fewer than 2 valid stocks")
    else:
        # Save the rolling window CSV
        window_path = os.path.join(rolling_dir, f'rolling_window_{rebal_str}_quicktest.csv')
        window_df.to_csv(window_path, index=False)
        print(f"✓ Saved rolling window to {window_path}\n")
        
        # Compute HRP weights using PERMNO as unique index
        returns = window_df[actual_window_cols].T  # Time x Assets
        returns.columns = window_df['PERMNO'].astype(str)  # Use PERMNO as unique columns (str)
        
        print(f"Computing HRP weights for {len(returns.columns)} stocks...")
        
        # Compute shrunk covariance using Ledoit-Wolf
        lw = LedoitWolf().fit(returns)
        cov_array = lw.covariance_
        cov = pd.DataFrame(cov_array, index=returns.columns, columns=returns.columns)
        
        # Compute correlation from shrunk cov
        std = np.sqrt(np.diag(cov_array))
        corr_array = cov_array / np.outer(std, std)
        corr = pd.DataFrame(corr_array, index=returns.columns, columns=returns.columns)
        
        # Distances
        dist = get_correlation_distance(corr)
        eucl_dist = get_euclidean_distance(dist)
        
        # Clustering
        link = sch.linkage(eucl_dist, method='single')
        
        # Seriation
        sort_ix = get_quasi_diag(link)
        sort_ix = returns.columns[sort_ix].tolist()  # List of str(PERMNO)
        
        # Weights
        hrp_weights = get_recursive_bisection(cov, sort_ix)
        
        # Map back to Company_Ticker
        permno_to_ticker = dict(zip(window_df['PERMNO'].astype(str), window_df['Company_Ticker']))
        
        # Store with Company_Ticker
        weights_df = pd.DataFrame({'PERMNO': hrp_weights.index, 'Weight': hrp_weights.values})
        weights_df['Company_Ticker'] = weights_df['PERMNO'].map(permno_to_ticker)
        weights_df = weights_df[['Company_Ticker', 'PERMNO', 'Weight']]
        weights_df['Rebalance_Date'] = rebal_str
        
        # Save weights
        weights_path = os.path.join(rolling_dir, f'hrp_weights_quicktest_{rebal_str}.csv')
        weights_df.to_csv(weights_path, index=False)
        print(f"✓ Saved HRP weights to {weights_path}\n")
        
        # Display summary statistics
        print("\n" + "="*60)
        print("WEIGHT STATISTICS")
        print("="*60)
        print(f"Total number of stocks: {len(weights_df)}")
        print(f"Sum of weights: {weights_df['Weight'].sum():.6f}")
        print(f"\nWeight distribution:")
        print(weights_df['Weight'].describe())
        print(f"\nTop 10 holdings:")
        print(weights_df.nlargest(10, 'Weight')[['Company_Ticker', 'Weight']])
        print(f"\nBottom 10 holdings:")
        print(weights_df.nsmallest(10, 'Weight')[['Company_Ticker', 'Weight']])

In [ ]:
# Cell 5: Visualize the Dendrogram (Optional)
import matplotlib.pyplot as plt

if 'link' in locals():
    plt.figure(figsize=(14, 7))
    sch.dendrogram(link, labels=returns.columns.tolist(), leaf_font_size=8)
    plt.title(f'Hierarchical Clustering Dendrogram - {rebal_str}')
    plt.xlabel('Stock PERMNO')
    plt.ylabel('Distance')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()
else:
    print("No clustering data available to plot.")

In [ ]:
# Cell 6: Visualize Weight Distribution
if 'weights_df' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histogram
    axes[0].hist(weights_df['Weight'], bins=50, edgecolor='black')
    axes[0].set_xlabel('Weight')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Distribution of HRP Weights')
    axes[0].grid(True, alpha=0.3)
    
    # Sorted weights plot
    sorted_weights = weights_df.sort_values('Weight', ascending=False).reset_index(drop=True)
    axes[1].plot(sorted_weights.index, sorted_weights['Weight'], marker='o', markersize=2, linestyle='-')
    axes[1].set_xlabel('Stock Rank')
    axes[1].set_ylabel('Weight')
    axes[1].set_title('Sorted HRP Weights')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No weights data available to plot.")